# Agentic AI MCP - Server Only

This notebook starts an MCP server that exposes tools.
Run this on the machine where you want to host the tools.

Another machine can connect to this server using the client notebook.

## Step 1: Setup

In [7]:
# install if needed
# !uv pip install agentic_ai_mcp==0.5.0

import agentic_ai_mcp
print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.5.0


## Step 2: Define Your Tools

In [8]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

def greet(name: str, times: int = 1) -> str:
    """Greet someone."""
    return ("Hello, " + name + "! ") * times

## Step 3: Create AgenticAI and Register Tools

Configure the host/port. 

In [9]:
from agentic_ai_mcp import AgenticAI

ai = AgenticAI(host="0.0.0.0", port=8888)

# register your tools
ai.register_tool(add)
ai.register_tool(multiply)
ai.register_tool(greet)

print(f"Registered tools: {ai.tools}")

Registered tools: ['add', 'multiply', 'greet']


## Step 4: Start the MCP Server

This will start the server in background. The server URL will be:
- Local: `http://127.0.0.1:8888/mcp`
- Remote: `http://<your-ip>:8888/mcp`

Give this URL to the client notebook.

In [17]:
# start server in background 
ai.run_mcp_server()

# get your IP for remote connections
import socket
hostname = socket.gethostname()
try:
    local_ip = socket.gethostbyname(hostname)
except:
    local_ip = "<your-ip>"

print(f"\nServer is ready!")
print(f"Local URL:  http://127.0.0.1:8888/mcp")
print(f"Remote URL: http://{local_ip}:8888/mcp")

INFO:     Started server process [69401]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)



Server is ready!
Local URL:  http://127.0.0.1:8888/mcp
Remote URL: http://127.0.1.1:8888/mcp
INFO:     127.0.0.1:35984 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35996 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:36004 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36006 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36012 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36028 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36034 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:36046 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36060 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36068 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36076 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36078 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36088 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:36098 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36102 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:36108 - "POST /mcp HTTP/1.1

[02/26/26 07:27:08] Error validating tool 'greet'                                                                  
                    ╭───────────────────────────── Traceback (most recent call last) ─────────────────────────────╮
                    │ /mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/env_caa/lib/python3.13/site-p │
                    │ ackages/fastmcp/server/server.py:982 in call_tool                                           │
                    │                                                                                             │
                    │ /mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/env_caa/lib/python3.13/site-p │
                    │ ackages/fastmcp/tools/tool.py:332 in _run                                                   │
                    │                                                                                             │
                    │                                   ... 4 frames hidden ...                                   │
                    │                                                                                             │
                    │ /mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/env_caa/lib/python3.13/site-p │
                    │ ackages/anyio/_backends/_asyncio.py:986 in run                                              │
                    │                                                                                             │
                    │    983 │   │   │   │   │   exception: BaseException | None = None                           │
                    │    984 │   │   │   │   │   threadlocals.current_cancel_scope = cancel_scope                 │
                    │    985 │   │   │   │   │   try:                                                             │
                    │ ❱  986 │   │   │   │   │   │   result = context.run(func, *args)                            │
                    │    987 │   │   │   │   │   except BaseException as exc:                                     │
                    │    988 │   │   │   │   │   │   exception = exc                                              │
                    │    989 │   │   │   │   │   finally:                                                         │
                    │                                                                                             │
                    │ /mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/env_caa/lib/python3.13/site-p │
                    │ ackages/pydantic/type_adapter.py:441 in validate_python                                     │
                    ╰─────────────────────────────────────────────────────────────────────────────────────────────╯
                    ValidationError: 1 validation error for call[greet]                                            
                    times                                                                                          
                      Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]       
                        For further information visit https://errors.pydantic.dev/2.12/v/int_type

INFO:     127.0.0.1:38614 - "DELETE /mcp HTTP/1.1" 200 OK


## Step 5: Stop the Server

Run this cell to stop the server when you're done.

In [16]:
ai.stop_mcp_server()